In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import os
from os import path

In [ ]:
cd '/content/drive/MyDrive/Grad Project/MIT'

/content/drive/MyDrive/Grad Project/MIT


In [ ]:
def get_labels():
  labels_path = 'turker_scores_full_interview.csv'
  data_frame = pd.read_csv(labels_path)
  data_frame = data_frame[data_frame['Worker'] == 'AGGR']
  return data_frame

In [ ]:
get_labels()

,Participant,Worker,Overall,RecommendHiring,Colleague,Engaged,Excited,EyeContact,Smiled,SpeakingRate,NoFillers,Friendly,Paused,EngagingTone,StructuredAnswers,Calm,NotStressed,Focused,Authentic,NotAwkward,Total
9,p1,AGGR,5.297316,5.106224,5.333004,5.541380,5.043890,5.866119,3.576160,4.865590,3.771665,5.254784,5.800468,5.147909,4.891580,5.351075,5.350760,5.845226,5.610513,5.477534,93.131196
19,p3,AGGR,4.414892,4.433070,5.010430,5.616076,5.601586,5.426861,6.062173,5.016286,3.332458,6.500329,4.791550,5.621231,3.912199,4.476537,5.521215,5.549829,6.014960,4.923550,92.225232
29,p4,AGGR,4.494494,4.530129,4.500707,5.494494,4.261343,3.857848,4.852150,4.738000,3.111445,5.369884,5.177447,4.392736,4.688379,5.065190,5.566084,5.324136,5.658364,4.452909,85.535740
39,p5,AGGR,5.457670,5.571558,5.772488,5.903057,4.707062,6.694276,3.920479,4.927181,5.881741,5.649119,5.886326,4.695523,5.582514,6.130488,5.916373,6.322086,6.020070,5.629838,100.667850
49,p6,AGGR,5.106512,4.831482,4.860595,5.020385,4.648259,4.331805,4.194227,4.174891,3.920329,4.495375,5.027823,4.261988,4.826245,5.828977,5.824390,5.619420,5.371858,5.295977,87.640538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,pp83,AGGR,6.045748,5.806617,6.347873,6.243224,5.806617,5.847779,6.483032,4.655062,4.838943,6.549279,5.764552,5.801601,4.878478,5.492810,5.391449,6.083629,6.559118,5.840547,104.436358
1348,pp84,AGGR,5.710073,6.020304,5.437203,5.749959,5.307262,5.564999,6.533606,4.656370,5.047772,6.529525,5.912420,5.625489,4.890114,5.112062,5.750670,5.988186,6.248887,5.169949,101.254849
1358,pp85,AGGR,5.626074,5.766592,5.791370,6.375623,5.584830,6.280263,5.050913,4.263639,4.097177,5.970750,5.643748,5.239474,4.918227,4.985095,4.998952,6.554104,6.372852,4.800594,98.320278
1368,pp86,AGGR,4.853881,4.700179,4.495230,5.513933,5.137644,5.892929,4.875432,4.645732,4.126127,5.022182,4.814251,5.208898,4.240302,5.648693,5.939815,5.078459,6.008339,5.154665,91.356689


In [ ]:
def get_input_features():
  features_path = 'prosodic_features.csv'
  data_frame = pd.read_csv(features_path)
  arr_features = data_frame.to_numpy()[: , 1:-1]
  return arr_features

In [ ]:
def average_5Qs():

  arr_features = get_input_features()
  Total_interviews = 138
  Total_questions = 5
  Total_data = []
  for i in range(Total_interviews):
    interview_data = []
    for feature_idx in range(arr_features.shape[1]):
      average_on = 0
      feature_sum = 0
      for j in range(Total_questions):
        feature_value = arr_features[i*Total_questions + j][feature_idx]
        if feature_value != '--undefined--':
          average_on = average_on + 1
          feature_sum = feature_sum + float(feature_value)

      interview_data.append(feature_sum / average_on)

    Total_data.append(np.array(interview_data))
    #print(str(i))

  
  interviews_arr = np.stack(Total_data , axis=0)
  print(interviews_arr.shape)
  return interviews_arr

# Training

In [ ]:
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm

In [ ]:
def set_data(trait = 'Overall'):
  
  all_interviews = average_5Qs()
  labels = get_labels()[trait]


  P_data , PP_data , P_labels , PP_labels = all_interviews[0:69 , : ] , all_interviews[69: , : ] , labels[0:69] , labels[69:]
  P_data , PP_data , P_labels , PP_labels = shuffle(P_data , PP_data , P_labels , PP_labels , random_state = 2)

  del(all_interviews)
  del(labels)

  train_data , test_data = np.concatenate((P_data[0:55 , : ] , PP_data[0:55 , : ])) , np.concatenate((P_data[55: , : ] , PP_data[55: , : ]))
  train_labels, test_labels = np.concatenate((P_labels[0:55] , PP_labels[0:55])), np.concatenate((P_labels[55:] , PP_labels[55:]))

  return train_data , test_data , train_labels , test_labels

In [ ]:
def training_SVR(trait = 'Overall'):
  
  train_data , test_data , train_labels , test_labels = set_data(trait)
  scaler = MinMaxScaler()
  scaler.fit(train_data)
  train_data = scaler.transform(train_data)
  test_data = scaler.transform(test_data)

  param_grid = [{'C': [1, 10, 100, 1000], 'kernel': ['linear' , 'rbf']}]

  model_svr = svm.SVR()
  grids = GridSearchCV(model_svr,param_grid,cv=10)

  
  grids.fit(train_data , train_labels)
  Y_predicted = grids.predict(test_data)
  correlation_res = pearsonr(Y_predicted , test_labels)
  print('--------------------------------' + trait + '--------------------------------')
  print(correlation_res)
  print(Y_predicted)

  return model_svr  

# random state = 0

In [ ]:
training_SVR('Overall')

(138, 58)
--------------------------------Overall--------------------------------
(0.2208113495744794, 0.2588303254703342)
[5.30093184 5.10471743 5.39520807 5.5351889  5.565052   5.218611
 5.3584196  4.72386892 5.34203818 5.3866171  4.81704188 5.176645
 4.8278551  5.01362146 5.67584524 5.29931191 5.36285195 5.42645123
 5.54768129 5.27287975 4.95975436 5.22466241 5.60585953 4.96537014
 5.18349719 5.08852763 4.24395196 5.6291391 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 58)
--------------------------------RecommendHiring--------------------------------
(0.12148535647172821, 0.5380137435852271)
[5.01869929 5.04004009 5.43793356 5.6826587  5.66699199 5.08415361
 5.45035951 4.74435054 5.02677904 5.118651   4.60018859 5.12652895
 4.69868217 4.83858322 5.46022586 5.23856069 5.20140223 5.56609208
 5.41890232 5.18819037 4.90895673 5.15006937 5.53344234 4.68111427
 5.05613494 4.87579358 4.15571634 5.7136104 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Friendly')

(138, 58)
--------------------------------Friendly--------------------------------
(0.2558015729303713, 0.18889737373889004)
[5.02288217 5.0296108  5.59945062 6.22991035 6.66076062 5.69050745
 6.57536688 4.56299976 5.4284315  5.50039276 5.35991604 5.99012402
 5.04696947 5.84436917 5.61400738 5.23672985 5.5287378  6.36535301
 5.85860495 5.59168815 6.3376584  5.19401141 5.50892646 5.55286448
 5.45801569 5.77623927 4.32921599 6.36373363]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Engaged')

(138, 58)
--------------------------------Engaged--------------------------------
(0.35387448862158294, 0.06468194650551869)
[5.31421011 5.73798619 5.93955522 5.83306415 6.20755486 5.71412409
 6.05606094 5.26086317 5.64323237 5.41037229 5.19740306 5.68412646
 5.15418955 5.67393072 5.83854604 5.91633118 5.95396501 6.0049957
 5.94394722 5.74386918 5.87420683 5.51824532 5.8993414  5.41757306
 5.71732639 5.59555807 4.41827142 6.04297531]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Excited')

(138, 58)
--------------------------------Excited--------------------------------
(0.48355981875562554, 0.009135138991087933)
[4.33415304 4.79927857 5.34066116 5.70920415 5.96220328 4.99187958
 5.68041989 4.36427794 4.77178195 4.61701869 4.98858764 5.67153294
 4.53349254 5.05930836 5.20050961 5.02768663 5.29772644 5.72418701
 5.24950703 5.14924259 5.3904437  4.8568114  5.14874859 4.5933597
 5.24812334 5.23328095 3.65598147 5.75934896]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# random state = 2

In [ ]:
training_SVR('Overall')

(138, 58)
--------------------------------Overall--------------------------------
(0.6241859713772901, 0.00038559963907957174)
[5.41812793 4.93608624 5.20164912 4.9163914  5.41074482 5.4040656
 4.81009777 5.17173591 4.13673961 4.95189194 5.19066289 4.7261029
 4.81336328 4.37602119 5.08889286 4.99016688 5.087717   4.50719744
 4.92581929 5.9960545  4.57129975 5.20481919 4.48411897 4.94674869
 5.56650429 4.48798718 5.1612904  4.74360207]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 58)
--------------------------------RecommendHiring--------------------------------
(0.6499341380464984, 0.00018163790064317954)
[5.08537158 4.77368503 5.2936261  4.72757582 5.15320048 5.17128568
 4.63989893 5.04000142 3.98602305 4.73808983 5.08542999 4.58925644
 4.73311091 4.42660455 4.93452535 4.8448032  5.08160176 4.3537385
 4.88401674 5.85378514 4.41828633 5.06714698 4.33139149 4.53999602
 5.54120228 4.3990633  5.06428224 4.7646145 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Friendly')

(138, 58)
--------------------------------Friendly--------------------------------
(0.6489980108740739, 0.00018690386530493023)
[5.47337351 5.46201663 6.01094661 5.1790933  5.70893802 6.00971677
 5.17302849 4.86193936 4.48722023 5.21535818 6.21910106 5.20587856
 5.60833656 5.17778151 5.6913337  5.14496691 5.7700845  4.14153355
 5.72907066 6.53718532 5.15273822 5.06159588 4.70943531 4.7946074
 6.45081276 5.19863004 5.47652006 5.07505976]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Engaged')

(138, 58)
--------------------------------Engaged--------------------------------
(0.638594025363749, 0.00025516613921591823)
[5.44416107 5.41499984 5.7223717  5.33601197 5.41767443 5.58796536
 5.27230128 5.68115921 4.68954098 5.43689395 5.65229663 5.24336672
 5.04001244 4.91760558 5.40934442 5.66022652 5.79973921 4.66823353
 5.34178745 6.13356396 5.04157056 5.77615578 4.85619923 4.97449239
 5.90549447 5.08769825 5.4027612  4.94550587]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Excited')

(138, 58)
--------------------------------Excited--------------------------------
(0.7427825139229088, 5.992863404026658e-06)
[4.85400624 4.79949527 5.42246145 4.7466179  4.82093442 5.15480018
 4.641787   4.93431103 3.42450551 4.64313878 5.17241674 4.45649821
 4.25366573 3.98381054 4.95224488 4.96527333 5.33153006 3.73415567
 4.72572203 5.77611537 4.50515662 5.13740254 3.83449802 4.28717653
 5.58860173 4.13791135 4.73657441 4.17954586]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('EngagingTone')

(138, 58)
--------------------------------EngagingTone--------------------------------
(0.7629818934431605, 2.34750603251627e-06)
[5.0518793  4.84264696 5.39577658 4.73794918 4.99365452 5.13501037
 4.63227802 4.57392925 3.65611398 4.79263254 5.37725192 4.47207176
 4.01833781 4.01228688 5.20498379 4.90409433 5.36178778 3.83262006
 5.08820782 5.99609024 4.42379497 4.84659496 4.05658855 4.65797202
 5.80732667 4.24073861 4.67106732 4.04633697]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# random state = 4

In [ ]:
training_SVR('Overall')

(138, 58)
--------------------------------Overall--------------------------------
(0.5583013425738763, 0.0020184632890074597)
[4.8877993  4.43600892 5.09697195 5.21579794 5.00996987 4.51387763
 5.10127547 4.98141085 5.12620604 5.01831471 5.2986486  5.13921308
 5.13893002 4.72620456 5.08632675 4.49095761 5.48984242 4.80379872
 4.88088195 4.44442916 5.29628358 5.14904968 5.14456971 5.3239368
 5.63963841 5.4076748  5.64005017 5.00506692]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 58)
--------------------------------RecommendHiring--------------------------------
(0.6124143054917076, 0.0005323876007475809)
[4.76466502 4.41784215 4.78887736 4.82165877 4.95761068 4.52251295
 4.87472636 4.68017368 4.95157295 4.63159645 4.98125875 4.99379214
 4.99769159 4.57259599 4.83284582 4.61018919 5.29356407 4.36968461
 4.67308191 4.46635408 4.99776474 4.74430642 4.87200068 5.11704787
 5.50849081 5.25682635 5.35233553 4.86453102]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Friendly')

(138, 58)
--------------------------------Friendly--------------------------------
(0.32049956617868997, 0.09635297540362478)
[4.6471353  5.22286209 5.5553844  5.57713651 4.74798242 5.15261416
 5.3983108  5.5907176  5.53726258 5.54757443 5.44456236 4.96287022
 5.46861692 5.14946209 5.03697156 5.34882911 5.80866286 5.23944897
 4.70111555 5.18286674 5.64579928 5.70292969 5.42937329 5.64394861
 5.48674782 5.1844033  5.81464479 5.67221271]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Engaged')

(138, 58)
--------------------------------Engaged--------------------------------
(0.6561145136242386, 0.00015004059926075755)
[5.39574166 4.94050011 5.19754741 5.59223985 5.31996219 5.15272425
 5.14375863 5.2111284  5.31215156 5.20053326 5.71914616 5.83220123
 5.50275079 5.37127783 5.45048446 4.94304618 5.65965186 5.14320901
 5.21168212 5.01092949 5.28108071 5.19323589 5.31189564 5.64592938
 6.04972509 6.07613352 5.77464331 5.67566667]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Excited')

(138, 58)
--------------------------------Excited--------------------------------
(0.6525526399945517, 0.00016759729484611797)
[4.3124064  4.09583871 4.94461152 4.96055348 4.54912273 4.0019209
 4.27724812 4.52849079 4.44501196 4.79724029 4.77287328 4.60489944
 4.85789277 4.65815362 4.56586598 4.06832859 5.2423189  4.24901627
 4.46765112 3.98488707 4.57355195 4.64047241 4.4472338  5.13255979
 5.19673572 4.93250673 5.09599131 5.05296942]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('EngagingTone')

(138, 58)
--------------------------------EngagingTone--------------------------------
(0.6261275598528894, 0.00036516422109109206)
[4.33954503 3.98379709 4.64390871 5.40292653 4.50128012 3.81218743
 4.37131853 4.59520786 4.50474796 5.09581841 4.95976501 4.61369747
 4.64641851 4.4448231  4.52718352 3.72041869 5.25893079 4.60481938
 4.23450574 3.76023112 4.73161422 4.79981717 4.6073378  5.32217846
 4.98566852 4.87720109 5.26006007 5.04543313]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# With Tuning

## random state = 0

In [ ]:
training_SVR('Overall')

(138, 58)
--------------------------------Overall--------------------------------
(0.2208113495744794, 0.2588303254703342)
[5.30093184 5.10471743 5.39520807 5.5351889  5.565052   5.218611
 5.3584196  4.72386892 5.34203818 5.3866171  4.81704188 5.176645
 4.8278551  5.01362146 5.67584524 5.29931191 5.36285195 5.42645123
 5.54768129 5.27287975 4.95975436 5.22466241 5.60585953 4.96537014
 5.18349719 5.08852763 4.24395196 5.6291391 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 58)
--------------------------------RecommendHiring--------------------------------
(0.12148535647172821, 0.5380137435852271)
[5.01869929 5.04004009 5.43793356 5.6826587  5.66699199 5.08415361
 5.45035951 4.74435054 5.02677904 5.118651   4.60018859 5.12652895
 4.69868217 4.83858322 5.46022586 5.23856069 5.20140223 5.56609208
 5.41890232 5.18819037 4.90895673 5.15006937 5.53344234 4.68111427
 5.05613494 4.87579358 4.15571634 5.7136104 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Friendly')

(138, 58)
--------------------------------Friendly--------------------------------
(0.2558015729303713, 0.18889737373889004)
[5.02288217 5.0296108  5.59945062 6.22991035 6.66076062 5.69050745
 6.57536688 4.56299976 5.4284315  5.50039276 5.35991604 5.99012402
 5.04696947 5.84436917 5.61400738 5.23672985 5.5287378  6.36535301
 5.85860495 5.59168815 6.3376584  5.19401141 5.50892646 5.55286448
 5.45801569 5.77623927 4.32921599 6.36373363]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Engaged')

(138, 58)
--------------------------------Engaged--------------------------------
(0.4379782095071076, 0.019750292334565787)
[5.38566402 5.74265684 5.83200271 5.56961931 6.31235171 5.72432085
 6.06917858 5.30065071 5.76947304 5.35512101 4.89038585 5.63625561
 5.0222579  5.44537391 6.08914771 5.9329634  5.82383288 5.6618347
 6.10983745 5.70749166 5.7608581  5.63958522 6.02738738 5.12187926
 5.79437778 5.51074598 4.1597918  5.82007971]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Excited')

(138, 58)
--------------------------------Excited--------------------------------
(0.34695301326196365, 0.0704728181934897)
[   4.69992978    5.18759531    5.1545514     5.97111504    5.713158
    5.45558167    5.47743276    4.41688274    5.25018169    5.10584627
    5.40565913    5.86938206 -109.75636939    5.4502273     5.470758
    5.56338998    5.07025658    5.67864556    5.2924491     5.2524579
    5.00243156    4.53618746    5.718038      4.62767063    5.21262721
    5.00342989    3.02788526    5.9020436 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('EngagingTone')

(138, 58)
--------------------------------EngagingTone--------------------------------
(0.3726292471667002, 0.050836693097916205)
[4.6879045  4.87830722 5.20978603 5.50690392 6.19802284 5.43939785
 5.64818549 4.31462474 5.21609977 4.92370611 4.90348025 4.9936422
 4.24007898 4.8507134  5.37955645 4.95507174 5.31976999 5.38485819
 6.02338335 5.73095964 5.35160765 4.57385473 5.28450002 4.95280516
 5.46819786 5.01975739 2.78632041 5.7394752 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

## random state = 2 

In [ ]:
training_SVR('Overall')

(138, 58)
--------------------------------Overall--------------------------------
(0.6241859713772901, 0.00038559963907957174)
[5.41812793 4.93608624 5.20164912 4.9163914  5.41074482 5.4040656
 4.81009777 5.17173591 4.13673961 4.95189194 5.19066289 4.7261029
 4.81336328 4.37602119 5.08889286 4.99016688 5.087717   4.50719744
 4.92581929 5.9960545  4.57129975 5.20481919 4.48411897 4.94674869
 5.56650429 4.48798718 5.1612904  4.74360207]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('RecommendHiring')

(138, 58)
--------------------------------RecommendHiring--------------------------------
(0.6499341380464984, 0.00018163790064317954)
[5.08537158 4.77368503 5.2936261  4.72757582 5.15320048 5.17128568
 4.63989893 5.04000142 3.98602305 4.73808983 5.08542999 4.58925644
 4.73311091 4.42660455 4.93452535 4.8448032  5.08160176 4.3537385
 4.88401674 5.85378514 4.41828633 5.06714698 4.33139149 4.53999602
 5.54120228 4.3990633  5.06428224 4.7646145 ]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Friendly')

(138, 58)
--------------------------------Friendly--------------------------------
(0.6489980108740739, 0.00018690386530493023)
[5.47337351 5.46201663 6.01094661 5.1790933  5.70893802 6.00971677
 5.17302849 4.86193936 4.48722023 5.21535818 6.21910106 5.20587856
 5.60833656 5.17778151 5.6913337  5.14496691 5.7700845  4.14153355
 5.72907066 6.53718532 5.15273822 5.06159588 4.70943531 4.7946074
 6.45081276 5.19863004 5.47652006 5.07505976]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Engaged')

(138, 58)
--------------------------------Engaged--------------------------------
(0.638594025363749, 0.00025516613921591823)
[5.44416107 5.41499984 5.7223717  5.33601197 5.41767443 5.58796536
 5.27230128 5.68115921 4.68954098 5.43689395 5.65229663 5.24336672
 5.04001244 4.91760558 5.40934442 5.66022652 5.79973921 4.66823353
 5.34178745 6.13356396 5.04157056 5.77615578 4.85619923 4.97449239
 5.90549447 5.08769825 5.4027612  4.94550587]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('Excited')

(138, 58)
--------------------------------Excited--------------------------------
(0.7969832070496127, 3.866304573636086e-07)
[4.55563268 5.04517098 5.84404852 4.62907062 4.72266806 5.1559177
 4.52434203 4.71797402 3.31221653 4.40416435 5.06108909 4.35181959
 4.10170923 4.11110851 4.76333917 4.99489384 5.70732039 3.5029919
 4.482968   5.82621075 4.43998999 5.02779979 3.8442501  4.16299868
 5.83720912 4.23737837 4.62291118 4.16267056]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
training_SVR('EngagingTone')

(138, 58)
--------------------------------EngagingTone--------------------------------
(0.7072379876268431, 2.5753689368530178e-05)
[4.8568878  4.83356674 5.42839588 4.50934357 4.89898894 4.95684827
 4.38131867 3.90203246 3.21481187 4.57798185 5.33880381 4.32165571
 3.56666303 3.83157513 4.97187556 4.47676188 5.34908769 3.09707579
 4.80356252 5.97520728 4.10732937 4.39392273 3.62932961 4.56820444
 5.8199338  4.21403899 4.45094414 3.79218014]


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)